# Plan

In [1]:
import vector
import yaml
import plotly.graph_objects as go

In [2]:
with open(r'cave.yml') as file:
    data = yaml.full_load(file)

In [3]:
points = data.get('points')

In [4]:
# Init data
for k in points.keys():
    
    print(f"Found point {k}")
    
    p = points.get(k)
    
    p['neighbour'] = []
    p['vector'] = vector.obj(x=p.get('x'), y=p.get('y'))

Found point A
Found point B
Found point C
Found point D
Found point E
Found point F
Found point G
Found point H


In [5]:
# populate neighbour
for d in data['distances']:
    # from
    points.get(d.get('from')).get('neighbour').append({
        'name': d.get('to'),
        'd': d.get('d')
    })
    # to
    points.get(d.get('to')).get('neighbour').append({
        'name': d.get('from'),
        'd': d.get('d')
    })

In [6]:
# data structure
points

{'A': {'x': 0.0,
  'y': 0.0,
  'x-lock': True,
  'y-lock': True,
  'neighbour': [{'name': 'B', 'd': 3.17},
   {'name': 'C', 'd': 4.257},
   {'name': 'D', 'd': 3.548},
   {'name': 'F', 'd': 2.891},
   {'name': 'G', 'd': 2.92},
   {'name': 'H', 'd': 1.749}],
  'vector': VectorObject2D(x=0.0, y=0.0)},
 'B': {'x': 3.17,
  'y': 0.0,
  'y-lock': True,
  'neighbour': [{'name': 'A', 'd': 3.17},
   {'name': 'C', 'd': 2.619},
   {'name': 'D', 'd': 5.999},
   {'name': 'F', 'd': 5.649}],
  'vector': VectorObject2D(x=3.17, y=0.0)},
 'C': {'x': 3.17,
  'y': 2.6,
  'neighbour': [{'name': 'A', 'd': 4.257},
   {'name': 'B', 'd': 2.619},
   {'name': 'D', 'd': 5.533},
   {'name': 'F', 'd': 5.533},
   {'name': 'G', 'd': 7.158}],
  'vector': VectorObject2D(x=3.17, y=2.6)},
 'D': {'x': -2.35,
  'y': 2.6,
  'neighbour': [{'name': 'A', 'd': 3.548},
   {'name': 'B', 'd': 5.999},
   {'name': 'C', 'd': 5.533},
   {'name': 'E', 'd': 0.855},
   {'name': 'F', 'd': 0.857},
   {'name': 'H', 'd': 5.007}],
  'vector': 

In [7]:
# apply correction in the oposite direction between two points
def force(a, b, d):
        
    ab = b - a
    
    return 0.2 * (abs(ab) - d) * ab.unit() 

In [8]:
def iterate():
    # Iteration loop
    for k in points.keys():

        # point
        p = points.get(k)

        #print(f"Point {k}")

        # initial force
        f = vector.obj(x=0, y=0)

        for n in p.get('neighbour'):

            #print('  Neighbour: ' + n.get('name'))

            f+= force(p.get('vector'), points.get(n.get('name')).get('vector'), n.get('d'))

        #print(f"force : {f}")
        
        # check if we have some coordinate lock
        if p.get('x-lock'):
            f.x = 0.0
        if p.get('y-lock'):
            f.y = 0.0

        # next point : apply coorection
        p['vector'] += f
    

In [9]:
# Total error between expected distances and mesured distance
def error():
    
    # Total Error
    e = 0.0
    
    for d in data.get('distances'):
        # point
        a = points.get(d.get('from')).get('vector')
        b = points.get(d.get('to')).get('vector')
        
        ab = b - a
        
        e+= abs(abs(ab) - d.get('d'))
        
    return e

In [10]:
def draw():
    
    xx = []
    yy = []

    # Iteration loop
    for k in points.keys():

        # point
        p = points.get(k)

        # add to list
        xx.append(p.get('vector').x)
        yy.append(p.get('vector').y)
        
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = xx,
        y = yy,
        fill="toself"
    ))

    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
      )

    fig.show()

# Compute

In [11]:
# Initial poisition
draw()

In [12]:
for i in range(50):
    iterate()

In [13]:
error()

0.17313711110863003

In [14]:
draw()

## Area

In [15]:
from shapely.geometry import Polygon

In [16]:
xx = []
yy = []

# Iteration loop
for k in points.keys():

    # point
    p = points.get(k)

    # add to list
    xx.append(p.get('vector').x)
    yy.append(p.get('vector').y)

In [17]:
pgon = Polygon(zip(xx, yy)) # Assuming the OP's x,y coordinates

In [18]:
pgon.area

18.686426063058903

In [19]:
for k in points.keys():
    # point
    p = points.get(k)
    x = round(p.get('vector').x, 3)
    y = round(p.get('vector').y, 3)
    # print
    print(f"{k}\t{x}\t{y}")

A	0.0	0.0
B	3.15	0.0
C	3.319	2.63
D	-2.214	2.735
E	-2.264	1.877
F	-2.179	1.877
G	-2.322	-1.769
H	0.028	-1.745
